# Agentic Appraisal System

## Overview

This notebook demonstrates an AI-powered automobile repair cost estimation system. The system processes First Notice of Loss (FNOL) data to automatically generate detailed repair cost estimates for collision-damaged vehicles.

- Extracts vehicle information from FNOL JSON documents
- Classifies vehicles into standard categories (Car, Truck, SUV, Van)
- Calculates repair costs based on industry-standard pricing tiers
- Applies cost multipliers for luxury brands and vehicle age
- Generates comprehensive JSON repair estimates

## Workflow

1. **FNOL Data Loading** - Load and parse First Notice of Loss JSON file
2. **Vehicle Classification** - Determine vehicle type for appropriate pricing tier
3. **Damage Assessment** - Identify all damaged components from FNOL data
4. **Cost Calculation** - Apply pricing matrix and multipliers to estimate repair costs
5. **Report Generation** - Create structured JSON estimate with detailed breakdown
6. **Agent Execution** - Strands agent autonomously invokes estimation tool and provides summary



In [8]:
%pip install -q -r requirements.txt --no-cache-dir --force-reinstall

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 1.2 requires nvidia-ml-py3==7.352.0, which is not installed.
dash 2.18.1 requires dash-core-components==2.0.0, which is not installed.
dash 2.18.1 requires dash-html-components==2.0.0, which is not installed.
dash 2.18.1 requires dash-table==5.0.0, which is not installed.
jupyter-ai 2.30.0 requires faiss-cpu!=1.8.0.post0,<2.0.0,>=1.8.0, which is not installed.
aiobotocore 2.21.1 requires botocore<1.37.2,>=1.37.0, but you have botocore 1.40.60 which is incompatible.
autogluon-multimodal 1.2 requires jsonschema<4.22,>=4.18, but you have jsonschema 4.25.1 which is incompatible.
autogluon-multimodal 1.2 requires nltk<3.9,>=3.4.5, but you have nltk 3.9.1 which is incompatible.
autogluon-multimodal 1.2 requires omegaconf<2.3.0,>=2.1.1, but you have omegaconf 2.3.0 which is incompatible.
autogluon-mu

**Please restart your environment, so it can reflect new versions!**

In [9]:
import IPython

IPython.Application.instance().kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

### Checking if `bedrock-agentcore-starter-toolkit` version is 0.1.21

In [1]:
!pip freeze | grep boto
!pip freeze | grep agentcore

aioboto3 @ file:///home/conda/feedstock_root/build_artifacts/aioboto3_1742196379442/work
aiobotocore @ file:///home/conda/feedstock_root/build_artifacts/aiobotocore_1741606508148/work
boto3==1.40.50
botocore==1.40.60
opentelemetry-instrumentation-boto==0.59b0
bedrock-agentcore==0.1.7
bedrock-agentcore-starter-toolkit==0.1.24


In [2]:
# Import libraries
import os
import json
import requests
import boto3
import time
from boto3.session import Session
from strands.tools import tool

# Get boto session
boto_session = Session()

### Create Code for the Agent
Create agents folder if it's not created.

In [3]:
![ ! -d "agents" ] && mkdir agents

In [4]:
%%writefile agents/appraisal-agent.py
import os
import logging
import asyncio
from mcp import stdio_client, StdioServerParameters
from strands import Agent
from strands.models import BedrockModel
from strands.tools import tool
from strands.multiagent.a2a import A2AServer
from strands.tools.mcp import MCPClient
import argparse
from fastapi import FastAPI
import uvicorn

# Standard library imports
import json
from datetime import datetime

# AWS SDK
import boto3


logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


# AWS Configuration
AWS_REGION = "us-east-1"
MODEL_ID = "anthropic.claude-sonnet-4-5-20250929-v1:0"

# Initialize Bedrock client to verify connectivity
bedrock_client = boto3.client(
    service_name="bedrock-runtime",
    region_name=AWS_REGION
)

print(f"✓ AWS Bedrock configured for region: {AWS_REGION}")
print(f"✓ Using model: {MODEL_ID}")


#Vehicle Type Classification

def classify_vehicle_type(make: str, model: str) -> str:
    """
    Classify vehicle into type category for pricing purposes.
    
    Args:
        make: Vehicle manufacturer (e.g., "Honda", "Ford")
        model: Vehicle model name (e.g., "Accord", "F-150")
        
    Returns:
        One of: "Car", "Truck", "SUV", "Van"
    """
    # Normalize inputs for case-insensitive matching
    model_lower = model.lower()
    
    # SUV classification - common SUV models
    suv_keywords = [
        'explorer', 'pilot', 'cr-v', 'crv', 'rav4', 'highlander',
        'tahoe', 'suburban', 'expedition', '4runner', 'pathfinder',
        'traverse', 'durango', 'grand cherokee', 'cherokee', 'wrangler',
        'rogue', 'murano', 'armada', 'sequoia', 'land cruiser',
        'cx-5', 'cx-9', 'outback', 'forester', 'ascent',
        'santa fe', 'tucson', 'palisade', 'telluride', 'sorento',
        'sportage', 'atlas', 'tiguan', 'touareg', 'x5', 'x3',
        'q5', 'q7', 'gx', 'rx', 'nx', 'xt5', 'xt6', 'escalade'
    ]
    
    # Truck classification - pickup trucks
    truck_keywords = [
        'f-150', 'f150', 'f-250', 'f250', 'f-350', 'f350',
        'silverado', 'sierra', 'ram', 'tundra', 'tacoma',
        'ranger', 'colorado', 'canyon', 'frontier', 'titan',
        'ridgeline', 'gladiator', 'maverick'
    ]
    
    # Van classification - minivans and cargo vans
    van_keywords = [
        'odyssey', 'sienna', 'pacifica', 'caravan', 'grand caravan',
        'transit', 'sprinter', 'promaster', 'metris', 'express',
        'savana', 'nv', 'quest', 'sedona', 'carnival'
    ]
    
    # Check for SUV
    for keyword in suv_keywords:
        if keyword in model_lower:
            return "SUV"
    
    # Check for Truck
    for keyword in truck_keywords:
        if keyword in model_lower:
            return "Truck"
    
    # Check for Van
    for keyword in van_keywords:
        if keyword in model_lower:
            return "Van"
    
    # Default to Car for sedans, coupes, hatchbacks, and unknown models
    return "Car"

# Pricing matrix: Component costs by vehicle type
# Format: {component: {vehicle_type: (min_cost, max_cost)}}
PRICING_MATRIX = {
    # Bumpers - Front and rear impact protection
    "front bumper": {
        "Car": (800, 1500),
        "Truck": (1200, 2000),
        "SUV": (1000, 1800),
        "Van": (900, 1600)
    },
    "rear bumper": {
        "Car": (800, 1500),
        "Truck": (1200, 2000),
        "SUV": (1000, 1800),
        "Van": (900, 1600)
    },
    
    # Hood - Engine compartment cover
    "hood": {
        "Car": (1000, 2000),
        "Truck": (1500, 3000),
        "SUV": (1200, 2500),
        "Van": (1100, 2200)
    },
    
    # Trunk - Rear storage compartment
    "trunk": {
        "Car": (1200, 2500),
        "Truck": (1800, 3500),
        "SUV": (1500, 3000),
        "Van": (1400, 2800)
    },
    
    # Doors - Per door pricing
    "door": {
        "Car": (1500, 3000),
        "Truck": (2000, 4000),
        "SUV": (1800, 3500),
        "Van": (1700, 3200)
    },
    
    # Side panels - Per panel pricing
    "side panel": {
        "Car": (1800, 3500),
        "Truck": (2500, 5000),
        "SUV": (2200, 4500),
        "Van": (2000, 4000)
    },
    
    # Fenders - Per fender pricing
    "fender": {
        "Car": (900, 2000),
        "Truck": (1400, 2800),
        "SUV": (1200, 2500),
        "Van": (1000, 2200)
    },
    
    # Headlights - Per light pricing
    "headlight": {
        "Car": (400, 800),
        "Truck": (600, 1200),
        "SUV": (500, 1000),
        "Van": (450, 900)
    },
    
    # Taillights - Per light pricing
    "taillight": {
        "Car": (300, 600),
        "Truck": (500, 800),
        "SUV": (400, 700),
        "Van": (350, 650)
    },
    
    # Grille - Front air intake
    "grille": {
        "Car": (400, 900),
        "Truck": (700, 1500),
        "SUV": (600, 1200),
        "Van": (500, 1000)
    },
    
    # Radiator - Cooling system
    "radiator": {
        "Car": (800, 1500),
        "Truck": (1200, 2500),
        "SUV": (1000, 2000),
        "Van": (900, 1800)
    },
    
    # Frame damage - Structural damage (most expensive)
    "frame": {
        "Car": (2000, 5000),
        "Truck": (3000, 8000),
        "SUV": (2500, 6500),
        "Van": (2200, 6000)
    }
}


def get_component_cost(component: str, vehicle_type: str) -> float:
    """
    Get the estimated cost for a damaged component based on vehicle type.
    Uses the midpoint of the cost range for the estimate.
    
    Args:
        component: Name of damaged component (e.g., "rear bumper", "hood")
        vehicle_type: Vehicle classification ("Car", "Truck", "SUV", "Van")
    
    Returns:
        Estimated cost as float (midpoint of range)
    """
    # Normalize component name to lowercase for matching
    component_lower = component.lower().strip()
    
    # Handle common variations in component naming
    component_mapping = {
        "front bumper": "front bumper",
        "rear bumper": "rear bumper",
        "bumper": "rear bumper",  # Default to rear if not specified
        "hood": "hood",
        "trunk": "trunk",
        "door": "door",
        "doors": "door",
        "side panel": "side panel",
        "panel": "side panel",
        "fender": "fender",
        "fenders": "fender",
        "headlight": "headlight",
        "headlights": "headlight",
        "taillight": "taillight",
        "taillights": "taillight",
        "tail light": "taillight",
        "tail lights": "taillight",
        "grille": "grille",
        "grill": "grille",
        "radiator": "radiator",
        "frame": "frame",
        "frame damage": "frame"
    }
    
    # Map component to standard name
    standard_component = component_mapping.get(component_lower, component_lower)
    
    # Look up pricing
    if standard_component in PRICING_MATRIX:
        min_cost, max_cost = PRICING_MATRIX[standard_component][vehicle_type]
        # Return midpoint of range
        return (min_cost + max_cost) / 2
    
    # If component not found, return 0 (will be handled in error reporting)
    return 0.0


#Repair Cost Estimation Tool
@tool
def estimate_repair_costs(
    vehicle_year: int,
    vehicle_make: str,
    vehicle_model: str,
    vehicle_vin: str,
    vehicle_mileage: int,
    damage_areas: list,
    damage_description: str,
    claim_number: str
) -> str:
    """
    Estimate repair costs for a collision-damaged vehicle based on vehicle type,
    damaged components, and applicable cost multipliers.
    
    This tool analyzes vehicle information and damage details to generate a comprehensive
    repair cost estimate. It classifies the vehicle into a pricing tier (Car, Truck, SUV, Van),
    calculates costs for each damaged component, applies multipliers for luxury brands and
    vehicle age, and returns a detailed JSON estimate.
    
    Args:
        vehicle_year: Year of vehicle manufacture (e.g., 2020)
        vehicle_make: Vehicle manufacturer name (e.g., "Honda", "Ford", "BMW")
        vehicle_model: Vehicle model name (e.g., "Accord", "F-150", "X5")
        vehicle_vin: Vehicle Identification Number (17-character alphanumeric)
        vehicle_mileage: Current vehicle mileage in miles (e.g., 35650)
        damage_areas: List of damaged components (e.g., ["Rear bumper", "Trunk", "Taillights"])
        damage_description: Detailed description of damage (e.g., "Rear bumper dented and paint scratched")
        claim_number: Insurance claim number for report metadata (e.g., "CL-2023-1156789")
    
    Returns:
        JSON string containing:
        - estimate_metadata: Claim number, estimate date, disclaimer
        - vehicle_information: Year, make, model, VIN, mileage, vehicle type classification
        - damage_assessment: Array of damaged components with individual costs and subtotal
        - cost_adjustments: Applied multipliers (luxury brand, age discount) with reasons
        - total_estimate: Final estimated repair cost in USD
    
    Example:
        >>> estimate = estimate_repair_costs(
        ...     vehicle_year=2020,
        ...     vehicle_make="Honda",
        ...     vehicle_model="Accord",
        ...     vehicle_vin="1HGCV2F35LA007149",
        ...     vehicle_mileage=35650,
        ...     damage_areas=["Rear bumper", "Trunk", "Taillights"],
        ...     damage_description="Rear-end collision damage",
        ...     claim_number="CL-2023-1156789"
        ... )
    """
    try:
        # Validate required parameters
        missing_fields = []
        if not vehicle_year:
            missing_fields.append("vehicle_year")
        if not vehicle_make:
            missing_fields.append("vehicle_make")
        if not vehicle_model:
            missing_fields.append("vehicle_model")
        if not vehicle_vin:
            missing_fields.append("vehicle_vin")
        if not damage_areas or len(damage_areas) == 0:
            missing_fields.append("damage_areas")
        if not claim_number:
            missing_fields.append("claim_number")
        
        if missing_fields:
            return json.dumps({
                "error": "Missing required fields",
                "missing_fields": missing_fields,
                "status": "validation_failed"
            })


        #Vehicle Type Classification
        #Classify the vehicle into one of four standard categories for pricing purposes:
        #Car: Sedans, coupes, hatchbacks (default category)
        #Truck: Pickup trucks (F-150, Silverado, Ram, etc.)
        #SUV: Sport utility vehicles (Explorer, Pilot, CR-V, RAV4, etc.)
        #Van: Minivans and cargo vans (Odyssey, Sienna, Transit, etc.)
        #The classification determines the base pricing tier for repair cost estimation.
    
        # Classify vehicle type for pricing
        vehicle_type = classify_vehicle_type(vehicle_make, vehicle_model)
        
        # Calculate cost multipliers
        multipliers = []
        cumulative_multiplier = 1.0
        
        # Luxury brand multiplier (1.3x)
        luxury_brands = [
            "lexus", "bmw", "mercedes-benz", "mercedes", "audi", "porsche",
            "jaguar", "land rover", "cadillac", "lincoln", "acura",
            "infiniti", "genesis"
        ]
        
        if vehicle_make.lower() in luxury_brands:
            luxury_multiplier = 1.3
            cumulative_multiplier *= luxury_multiplier
            multipliers.append({
                "type": "luxury_brand",
                "factor": luxury_multiplier,
                "reason": f"{vehicle_make} is a luxury brand with higher parts and labor costs"
            })
        
        # Age discount multiplier (0.85x for vehicles older than 10 years)
        current_year = datetime.now().year
        vehicle_age = current_year - vehicle_year
        
        if vehicle_age > 10:
            age_multiplier = 0.85
            cumulative_multiplier *= age_multiplier
            multipliers.append({
                "type": "age_discount",
                "factor": age_multiplier,
                "reason": f"Vehicle is {vehicle_age} years old (older than 10 years), parts costs are lower"
            })
        
        # Calculate costs for each damaged component
        damaged_components = []
        subtotal = 0.0
        
        for component in damage_areas:
            base_cost = get_component_cost(component, vehicle_type)
            
            if base_cost == 0.0:
                # Component not found in pricing matrix, skip it
                continue
            
            # Apply multipliers to component cost
            adjusted_cost = base_cost * cumulative_multiplier
            subtotal += adjusted_cost
            
            damaged_components.append({
                "component": component,
                "damage_description": damage_description,
                "estimated_cost": round(adjusted_cost, 2)
            })
        
        # Calculate adjustment total (difference from base subtotal)
        base_subtotal = subtotal / cumulative_multiplier if cumulative_multiplier != 1.0 else subtotal
        adjustment_total = subtotal - base_subtotal
        
        # Generate JSON output
        estimate = {
            "estimate_metadata": {
                "claim_number": claim_number,
                "estimate_date": datetime.now().isoformat(),
                "disclaimer": "This is a preliminary estimate. Actual repair costs may vary based on hidden damage, parts availability, and labor rates."
            },
            "vehicle_information": {
                "year": vehicle_year,
                "make": vehicle_make,
                "model": vehicle_model,
                "vin": vehicle_vin,
                "mileage": vehicle_mileage,
                "vehicle_type": vehicle_type
            },
            "damage_assessment": {
                "damaged_components": damaged_components,
                "subtotal": round(subtotal, 2)
            },
            "cost_adjustments": {
                "multipliers": multipliers,
                "adjustment_total": round(adjustment_total, 2)
            },
            "total_estimate": {
                "amount": round(subtotal, 2),
                "currency": "USD"
            }
        }
        
        return json.dumps(estimate, indent=2)
        
    except Exception as e:
        return json.dumps({
            "error": "Cost calculation failed",
            "details": str(e),
            "status": "calculation_failed"
        })



# Initialize Bedrock model
model = BedrockModel(
    model_id="us.anthropic.claude-sonnet-4-20250514-v1:0",
    region_name="us-east-1"
)

#Agent Instructions

agent_instructions="""You are an expert automotive claims adjuster specializing in repair cost estimation.
    
When provided with vehicle damage information, you should:
1. Use the estimate_repair_costs tool to generate detailed cost estimates and return JSON estimate
2. Analyze the returned JSON estimate carefully
3. Provide a clear, professional summary of the repair costs
4. Highlight key factors affecting the estimate (vehicle type, luxury brand, age, etc.)
5. Explain the breakdown of costs by damaged component
6. Note any cost adjustments and their reasons
7. Present the total estimated repair cost prominently

Your responses should be:
- Professional and authoritative
- Clear and easy to understand for insurance adjusters
- Detailed enough to justify the estimate
- Transparent about the preliminary nature of the estimate
- Include a section to provide the full JSON estimate

Always remind users that this is a preliminary estimate and actual costs may vary based on hidden damage, parts availability, and labor rates.
"""

# Create agent with repair cost estimation tool
agent = Agent(
    name="Repair Cost Estimator",
    description="A Single agent wit Appraisal tools capabilities",
    model=model,
    tools=[estimate_repair_costs],
    system_prompt=agent_instructions
)

################# A2A ################
app = FastAPI()
runtime_url = os.environ.get('AGENTCORE_RUNTIME_URL', 'http://127.0.0.1:9000/')
host, port = "0.0.0.0", 9000

a2a_server = A2AServer(
    agent=agent,
    http_url=runtime_url,
    serve_at_root=True,
    
)

@app.get("/ping")
def ping():
    return {"status": "healthy"}

# @app.on_event("startup")
# async def startup_event():
#     """Initialize MCP client on startup"""
#     await setup_agent_tools()

app.mount("/", a2a_server.to_fastapi_app())



if __name__ == "__main__":
    uvicorn.run(app, host=host, port=port)

################# A2A ################

################# Normal Config ################

# def strands_agent_bedrock(payload):
#     """
#     Invoke the agent with a payload
#     """
#     user_input = payload.get("prompt")
#     print (user_input)
#     response = agent(user_input)
#     return response.message['content'][0]['text']

# if __name__ == "__main__":
#     parser = argparse.ArgumentParser()
#     parser.add_argument("payload", type=str)
#     args = parser.parse_args()
#     response = strands_agent_bedrock(json.loads(args.payload))

################# Normal Config ################



Overwriting agents/appraisal-agent.py


In [5]:
# Load FNOL JSON data
with open('data/FNOL.json', 'r') as f:
    fnol_data = json.load(f)

# Extract vehicle information
insured_vehicle = fnol_data['vehicle']['insuredVehicle']
vehicle_year = insured_vehicle['year']
vehicle_make = insured_vehicle['make']
vehicle_model = insured_vehicle['model']
vehicle_vin = insured_vehicle['vin']
vehicle_mileage = insured_vehicle['mileage']

# Extract damage information
damage_info = insured_vehicle['damage']
damage_areas = damage_info['areas']
damage_description = damage_info['description']

# Extract claim number for report metadata
claim_number = fnol_data['fnol']['claimNumber']

# print (insured_vehicle)

In [6]:
# Create user prompt with vehicle and damage information
user_prompt = f"""
Please estimate the repair costs for the following vehicle damage claim:

**Claim Information:**
- Claim Number: {claim_number}
- Incident Date: {fnol_data['incident']['date']}
- Incident Description: {fnol_data['incident']['description']}

**Vehicle Information:**
- Year: {vehicle_year}
- Make: {vehicle_make}
- Model: {vehicle_model}
- VIN: {vehicle_vin}
- Mileage: {vehicle_mileage:,} miles

**Damage Assessment:**
- Damaged Areas: {', '.join(damage_areas)}
- Damage Description: {damage_description}
- Vehicle Drivable: {insured_vehicle['drivable']}

Please provide a detailed repair cost estimate including:
1. Cost breakdown by damaged component
2. Any applicable cost adjustments (luxury brand, vehicle age, etc.)
3. Total estimated repair cost
4. Professional summary and recommendations
"""

print("=" * 80)
print("USER PROMPT CREATED")
print("=" * 80)
print(user_prompt)
print("=" * 80)
print("\n📝 Prompt ready for agent execution")
print("\n" + "=" * 80)

USER PROMPT CREATED

Please estimate the repair costs for the following vehicle damage claim:

**Claim Information:**
- Claim Number: CL-2023-1156789
- Incident Date: 2023-11-19
- Incident Description: While stopped at a red light, my vehicle was rear-ended by another car. The impact pushed my vehicle forward about 5 feet. I pulled over to the side of the road to exchange information with the other driver.

**Vehicle Information:**
- Year: 2020
- Make: Honda
- Model: Accord
- VIN: 1HGCV2F35LA007149
- Mileage: 35,650 miles

**Damage Assessment:**
- Damaged Areas: Rear bumper, Trunk, Taillights, Possible frame
- Damage Description: Rear bumper dented and paint scratched, trunk lid misaligned, possible frame damage, brake lights damaged
- Vehicle Drivable: False

Please provide a detailed repair cost estimate including:
1. Cost breakdown by damaged component
2. Any applicable cost adjustments (luxury brand, vehicle age, etc.)
3. Total estimated repair cost
4. Professional summary and reco

In [7]:
# # !python agents/appraisal-agent.py '{"prompt": user_prompt}'

# # Create proper JSON payload
# payload = json.dumps({"prompt": user_prompt})

# # Run the command with proper escaping
# !python agents/appraisal-agent.py '{payload}'

In [8]:
# # Run the agent
# response = agent(user_prompt)

# print("\n" + "=" * 80)
# print("✓ Agent execution completed successfully")
# print("=" * 80)

In [9]:
# # Display agent's response
# print("=" * 80)
# print("APPRAISAL AGENT - SUMMARY")
# print("=" * 80)
# print()
# print(response)
# print()
# print("=" * 80)

In [11]:
from helpers.utils import setup_cognito_user_pool, reauthenticate_user

print("Setting up Amazon Cognito user pool...")
cognito_config = (
    setup_cognito_user_pool()
)  # You'll get your bearer token from this output cell.
print("Cognito setup completed ✓")

Setting up Amazon Cognito user pool...
Pool id: us-east-1_3yiuuvNmy
Discovery URL: https://cognito-idp.us-east-1.amazonaws.com/us-east-1_3yiuuvNmy/.well-known/openid-configuration
Client ID: 1lvtmj74pvs7oa07u3abe4cg4p
Bearer Token: eyJraWQiOiJMUnZVV2ZMQXNEXC9UQU4xVTdQRkpoRDA1blRFUDdDVkdIZlFXenhWc0VLUT0iLCJhbGciOiJSUzI1NiJ9.eyJzdWIiOiJmNDk4MTQ4OC1jMGIxLTcwMjYtOWFmNy04NmM3Y2JhN2NkMmUiLCJpc3MiOiJodHRwczpcL1wvY29nbml0by1pZHAudXMtZWFzdC0xLmFtYXpvbmF3cy5jb21cL3VzLWVhc3QtMV8zeWl1dXZObXkiLCJjbGllbnRfaWQiOiIxbHZ0bWo3NHB2czdvYTA3dTNhYmU0Y2c0cCIsIm9yaWdpbl9qdGkiOiI2ZTQzYTY1Mi0zZTcwLTRhNzYtODI0OS0yMWFjOTU2YjBkZDgiLCJldmVudF9pZCI6Ijk2YjAyMWEyLTViMTYtNDBiNi04NmI0LWMyN2JhNmNlMWU0NCIsInRva2VuX3VzZSI6ImFjY2VzcyIsInNjb3BlIjoiYXdzLmNvZ25pdG8uc2lnbmluLnVzZXIuYWRtaW4iLCJhdXRoX3RpbWUiOjE3NjE2NTgxMTcsImV4cCI6MTc2MTY2MTcxNywiaWF0IjoxNzYxNjU4MTE3LCJqdGkiOiI0YzBkMjE4ZC0wNGRlLTQwYjItYWY4Ny0yZGFjZTI3YjQ1YmIiLCJ1c2VybmFtZSI6InRlc3R1c2VyIn0.pkQQ03hOtlBbp9YvKPdXQep2vRzO63eqPSfhrZ1psY3sGMiWp3kUPzHiR54_QZw02OjB7nD02Q0

In [13]:
from helpers.utils import create_agentcore_runtime_execution_role, AWS_DOCS_ROLE_NAME

execution_role_arn_appraisal = create_agentcore_runtime_execution_role(AWS_DOCS_ROLE_NAME)

ℹ️ Role AWSDocsAssistantBedrockAgentCoreRole-us-east-1 already exists
Role ARN: arn:aws:iam::161615149547:role/AWSDocsAssistantBedrockAgentCoreRole-us-east-1


In [14]:
from bedrock_agentcore_starter_toolkit import Runtime

agentcore_runtime_appraisal_agent = Runtime()
appraisal_agent_name="aws_appraisal_assistant"

region = boto_session.region_name

# Configure the deployment
response_appraisal_agent = agentcore_runtime_appraisal_agent.configure(
    entrypoint="agents/appraisal-agent.py",
    execution_role=execution_role_arn_appraisal,
    auto_create_ecr=True,
    requirements_file="agents/requirements.txt",
    region=region,
    agent_name=appraisal_agent_name,
    authorizer_configuration={
        "customJWTAuthorizer": {
            "allowedClients": [cognito_config.get("client_id")],
            "discoveryUrl": cognito_config.get("discovery_url"),
        }
    },
    protocol="A2A",
)

print("Configuration completed:", response_appraisal_agent)

Entrypoint parsed: file=/home/sagemaker-user/Multi-Agent-Collaboration/hackathon_ClaimsAdjudication/agents/appraisal-agent.py, bedrock_agentcore_name=appraisal-agent
Memory configured with STM only
Configuring BedrockAgentCore agent: aws_appraisal_assistant


💡 No container engine found (Docker/Finch/Podman not installed)

✓ Default deployment uses CodeBuild (no container engine needed), For local builds, install Docker, Finch, or 
Podman

Will create new memory with mode: STM_ONLY
Memory configuration: Short-term memory only


⚠️ Platform mismatch: Current system is 'linux/amd64' but Bedrock AgentCore requires 'linux/arm64', so local builds
won't work.
Please use default launch command which will do a remote cross-platform build using code build.For deployment other
options and workarounds, see: 
https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/getting-started-custom.html

Generated .dockerignore
Generated Dockerfile: Dockerfile
Generated .dockerignore: /home/sagemaker-user/Multi-Agent-Collaboration/hackathon_ClaimsAdjudication/.dockerignore
Setting 'aws_appraisal_assistant' as default agent
Bedrock AgentCore configured: /home/sagemaker-user/Multi-Agent-Collaboration/hackathon_ClaimsAdjudication/.bedrock_agentcore.yaml


Configuration completed: config_path=PosixPath('/home/sagemaker-user/Multi-Agent-Collaboration/hackathon_ClaimsAdjudication/.bedrock_agentcore.yaml') dockerfile_path=PosixPath('/home/sagemaker-user/Multi-Agent-Collaboration/hackathon_ClaimsAdjudication/Dockerfile') dockerignore_path=PosixPath('/home/sagemaker-user/Multi-Agent-Collaboration/hackathon_ClaimsAdjudication/.dockerignore') runtime='None' region='us-east-1' account_id='161615149547' execution_role='arn:aws:iam::161615149547:role/AWSDocsAssistantBedrockAgentCoreRole-us-east-1' ecr_repository=None auto_create_ecr=True memory_id=None


In [15]:
launch_result_appraisal = agentcore_runtime_appraisal_agent.launch()
print("Launch completed:", launch_result_appraisal.agent_arn)

appraisal_agent_arn = launch_result_appraisal.agent_arn

🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
   • Build ARM64 containers in the cloud with CodeBuild
   • No local Docker required
💡 Available deployment modes:
   • runtime.launch()                           → CodeBuild (current)
   • runtime.launch(local=True)                 → Local development
   • runtime.launch(local_build=True)           → Local build + cloud deploy (NEW)
Creating memory resource for agent: aws_appraisal_assistant
✅ MemoryManager initialized for region: us-east-1
Creating new STM-only memory...
Created memory: aws_appraisal_assistant_mem-mbe9Un7lF8
Memory created but flag was False - correcting to True
✅ New memory created: aws_appraisal_assistant_mem-mbe9Un7lF8 (provisioning in background)
Starting CodeBuild ARM64 deployment for agent 'aws_appraisal_assistant' to account 161615149547 (us-east-1)
Setting up AWS resources (ECR repository, execution roles)...
Getting or creating ECR repository for agent: aws_appraisal_assistant
✅ ECR repository avai

Repository doesn't exist, creating new ECR repository: bedrock-agentcore-aws_appraisal_assistant


Using execution role from config: arn:aws:iam::161615149547:role/AWSDocsAssistantBedrockAgentCoreRole-us-east-1
Preparing CodeBuild project and uploading source...
Getting or creating CodeBuild execution role for agent: aws_appraisal_assistant
Role name: AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-09b58e53de
CodeBuild role doesn't exist, creating new role: AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-09b58e53de
Creating IAM role: AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-09b58e53de
✓ Role created: arn:aws:iam::161615149547:role/AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-09b58e53de
Attaching inline policy: CodeBuildExecutionPolicy to role: AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-09b58e53de
✓ Policy attached: CodeBuildExecutionPolicy
Waiting for IAM role propagation...
CodeBuild execution role creation complete: arn:aws:iam::161615149547:role/AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-09b58e53de
Using dockerignore.template with 45 patterns for zip filtering
Uploaded source t

Launch completed: arn:aws:bedrock-agentcore:us-east-1:161615149547:runtime/aws_appraisal_assistant-vUe2p2FZfq


In [16]:
status_response = agentcore_runtime_appraisal_agent.status()
status = status_response.endpoint["status"]

print(f"Final status: {status}")

✅ MemoryManager initialized for region: us-east-1
🔎 Retrieving memory resource with ID: aws_appraisal_assistant_mem-mbe9Un7lF8...
  Found memory: aws_appraisal_assistant_mem-mbe9Un7lF8
Retrieved Bedrock AgentCore status for: aws_appraisal_assistant


Final status: READY


In [17]:
bearer_token = reauthenticate_user(
    cognito_config.get("client_id"), 
    cognito_config.get("client_secret")
)

In [18]:
import logging
from uuid import uuid4
from urllib.parse import quote

logging.basicConfig(level=logging.ERROR)
logger = logging.getLogger(__name__)

def fetch_agent_card(agent_arn):
    # URL encode the agent ARN
    escaped_agent_arn = quote(agent_arn, safe='')

    # Construct the URL
    url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{escaped_agent_arn}/invocations/.well-known/agent-card.json"
    logger.info(url)
    # Generate a unique session ID
    session_id = str(uuid4())
    logger.info(f"Generated session ID: {session_id}")

    # Set headers
    headers = {
        'Accept': '*/*',
        'Authorization': f'Bearer {bearer_token}',
        'X-Amzn-Bedrock-AgentCore-Runtime-Session-Id': session_id,
        'X-Amzn-Trace-Id': f'aws_docs_assistant_{session_id}'
    }

    try:
        # Make the request
        response = requests.get(url, headers=headers)
        response.raise_for_status()

        # Parse and pretty print JSON
        agent_card = response.json()
        logger.info(json.dumps(agent_card, indent=2))

        return agent_card

    except requests.exceptions.RequestException as e:
        logger.error(f"Error fetching agent card: {e}")
        return None

In [19]:
fetch_agent_card(appraisal_agent_arn)

{'capabilities': {'streaming': True},
 'defaultInputModes': ['text'],
 'defaultOutputModes': ['text'],
 'description': 'A Single agent wit Appraisal tools capabilities',
 'name': 'Repair Cost Estimator',
 'preferredTransport': 'JSONRPC',
 'protocolVersion': '0.3.0',
 'skills': [{'description': 'Estimate repair costs for a collision-damaged vehicle based on vehicle type,\ndamaged components, and applicable cost multipliers.\n\nThis tool analyzes vehicle information and damage details to generate a comprehensive\nrepair cost estimate. It classifies the vehicle into a pricing tier (Car, Truck, SUV, Van),\ncalculates costs for each damaged component, applies multipliers for luxury brands and\nvehicle age, and returns a detailed JSON estimate.\n\nArgs:\n    vehicle_year: Year of vehicle manufacture (e.g., 2020)\n    vehicle_make: Vehicle manufacturer name (e.g., "Honda", "Ford", "BMW")\n    vehicle_model: Vehicle model name (e.g., "Accord", "F-150", "X5")\n    vehicle_vin: Vehicle Identific

#### Test agents

Now, let's invoke the first agent, using A2A:

In [20]:
import asyncio
import logging
import os
from uuid import uuid4

import httpx
from a2a.client import A2ACardResolver, ClientConfig, ClientFactory
from a2a.types import Message, Part, Role, TextPart

logging.basicConfig(level=logging.ERROR)
logger = logging.getLogger(__name__)

DEFAULT_TIMEOUT = 300  # set request timeout to 5 minutes

def format_agent_response(response):
    """Extract and format agent response for human readability."""
    # Get the main response text from artifacts
    if response.artifacts and len(response.artifacts) > 0:
        artifact = response.artifacts[0]
        if artifact.parts and len(artifact.parts) > 0:
            return artifact.parts[0].root.text
    
    # Fallback: concatenate all agent messages from history
    agent_messages = [
        msg.parts[0].root.text 
        for msg in response.history 
        if msg.role.value == 'agent' and msg.parts
    ]
    return ''.join(agent_messages)


def create_message(*, role: Role = Role.user, text: str) -> Message:
    return Message(
        kind="message",
        role=role,
        parts=[Part(TextPart(kind="text", text=text))],
        message_id=uuid4().hex,
    )

async def send_sync_message(agent_arn, message: str):
    # Get runtime URL from environment variable
    escaped_agent_arn = quote(agent_arn, safe='')

    # Construct the URL
    runtime_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{escaped_agent_arn}/invocations/"
    
    # Generate a unique session ID
    session_id = str(uuid4())
    print(f"Generated session ID: {session_id}")

    # Add authentication headers for AgentCore
    headers = {"Authorization": f"Bearer {bearer_token}",
              'X-Amzn-Bedrock-AgentCore-Runtime-Session-Id': session_id}
        
    async with httpx.AsyncClient(timeout=DEFAULT_TIMEOUT, headers=headers) as httpx_client:
        # Get agent card from the runtime URL
        resolver = A2ACardResolver(httpx_client=httpx_client, base_url=runtime_url)
        agent_card = await resolver.get_agent_card()
        print(agent_card)

        # Agent card contains the correct URL (same as runtime_url in this case)
        # No manual override needed - this is the path-based mounting pattern

        # Create client using factory
        config = ClientConfig(
            httpx_client=httpx_client,
            streaming=False,  # Use non-streaming mode for sync response
        )
        factory = ClientFactory(config)
        client = factory.create(agent_card)

        # Create and send message
        msg = create_message(text=message)

        # With streaming=False, this will yield exactly one result
        async for event in client.send_message(msg):
            if isinstance(event, Message):
                logger.info(event.model_dump_json(exclude_none=True, indent=2))
                return event
            elif isinstance(event, tuple) and len(event) == 2:
                # (Task, UpdateEvent) tuple
                task, update_event = event
                logger.info(f"Task: {task.model_dump_json(exclude_none=True, indent=2)}")
                if update_event:
                    logger.info(f"Update: {update_event.model_dump_json(exclude_none=True, indent=2)}")
                return task
            else:
                # Fallback for other response types
                logger.info(f"Response: {str(event)}")
                return event

In [21]:
result = await send_sync_message(appraisal_agent_arn, user_prompt)
formatted_output = format_agent_response(result)
print(formatted_output)

Generated session ID: d9a8bc51-8cc5-4e25-9259-e8d25f2739bc
additional_interfaces=None capabilities=AgentCapabilities(extensions=None, push_notifications=None, state_transition_history=None, streaming=True) default_input_modes=['text'] default_output_modes=['text'] description='A Single agent wit Appraisal tools capabilities' documentation_url=None icon_url=None name='Repair Cost Estimator' preferred_transport='JSONRPC' protocol_version='0.3.0' provider=None security=None security_schemes=None signatures=None skills=[AgentSkill(description='Estimate repair costs for a collision-damaged vehicle based on vehicle type,\ndamaged components, and applicable cost multipliers.\n\nThis tool analyzes vehicle information and damage details to generate a comprehensive\nrepair cost estimate. It classifies the vehicle into a pricing tier (Car, Truck, SUV, Van),\ncalculates costs for each damaged component, applies multipliers for luxury brands and\nvehicle age, and returns a detailed JSON estimate.\n